<a href="https://colab.research.google.com/github/Atfssene/FRASA/blob/main/Text_Summarization_Model_FRASA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Summarization Model

In this notebook, we will create a model for text summarization task. TextRank and SumBasic will be our feature extraction from senteces to create a weights that will be feeded to a neural networks. Let's start!

## Import library

In [1]:
import nltk
nltk.download('punkt')
!pip install fasttext

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
from nltk.tokenize import sent_tokenize
import numpy as np
import gzip
import networkx as nx
import tensorflow as tf
import pandas as pd
import heapq
import fasttext
import fasttext.util

## Read data

In [4]:
train = tf.keras.utils.get_file('train.csv', 'https://raw.githubusercontent.com/Atfssene/FRASA/main/Text%20Summarization/train.csv')
test = tf.keras.utils.get_file('test.csv', 'https://raw.githubusercontent.com/Atfssene/FRASA/main/Text%20Summarization/test.csv')

df_train = pd.read_csv(train)
df_test = pd.read_csv(test)
df_train.info()
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15012 entries, 0 to 15011
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   labels      15012 non-null  object
 1   paragraphs  15012 non-null  object
 2   summary     15012 non-null  object
 3   clean_text  15012 non-null  object
dtypes: object(4)
memory usage: 469.2+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3762 entries, 0 to 3761
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   labels      3762 non-null   object
 1   paragraphs  3762 non-null   object
 2   summary     3762 non-null   object
dtypes: object(3)
memory usage: 88.3+ KB


## Create TextRank

Download Indonesian word vector

In [9]:
# !wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.id.300.vec.gz
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.id.300.bin.gz

--2021-05-15 14:22:40--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.id.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 172.67.9.4, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4507049071 (4.2G) [application/octet-stream]
Saving to: ‘cc.id.300.bin.gz’

cc.id.300.bin.gz    100%[===================>]   4.20G  37.0MB/s    in 1m 58s  

2021-05-15 14:24:38 (36.6 MB/s) - ‘cc.id.300.bin.gz’ saved [4507049071/4507049071]



Unzipping...

In [13]:
# !gunzip cc.id.300.vec.gz
!gunzip cc.id.300.bin.gz

Count the words embeddings

In [ ]:
ft = fasttext.load_model('cc.id.300.bin')
print(ft.get_dimension())

fasttext.util.reduce_model(ft, 100)
print(ft.get_dimension())

300


In [ ]:

# ft.get_dimension()
# fasttext.util.reduce_model(ft, 100)

In [20]:
ft.get_dimension()

300

In [1]:
ft.get_nearest_neighbors('dokter')

NameError: ignored

In [15]:
word_embeddings = {}
f = open('/content/cc.id.300.vec', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

len(word_embeddings)

2000000

In [1]:
!cp /content/cc.id.300.bin /content/drive/MyDrive/model_summarization/cc.id.300.bin
!cp /content/cc.id.300.vec /content/drive/MyDrive/model_summarization/cc.id.300.vec

## Create SumBasic